In [115]:
import pandas as pd
import numpy as np
from scipy import sparse
import seaborn as sns
import matplotlib.pyplot as plt
import sys

from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity

pd.set_option('display.max_colwidth', None)

In [2]:
# read in board game ratings
df = pd.read_csv('./data/bgg-15m-reviews.csv', index_col = 0)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15823269 entries, 0 to 15823268
Data columns (total 5 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user     object 
 1   rating   float64
 2   comment  object 
 3   ID       int64  
 4   name     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 724.3+ MB


In [29]:
df[df['name'].str.contains('Pandemic')]['name'].value_counts()

Pandemic                              96234
Pandemic Legacy: Season 1             38376
Pandemic Legacy: Season 2             10614
Pandemic: The Cure                     8941
Pandemic: Iberia                       7775
Pandemic: Reign of Cthulhu             7265
Pandemic: Fall of Rome                 3213
Pandemic: Contagion                    2997
Pandemic: Rising Tide                  1845
Pandemic: Rapid Response               1325
Pandemic: Hot Zone – North America      238
Pandemic Legacy: Season 0                98
Name: name, dtype: int64

In [4]:
df.shape

(15823269, 5)

In [6]:
df.drop(columns = ['comment', 'ID'], inplace = True)

In [9]:
df.user.nunique()

351048

In [10]:
df.name.nunique()

18984

In [11]:
counts = df.groupby('name')['rating'].count()

In [12]:
users = df.groupby('user')['rating'].count()

In [13]:
users = pd.DataFrame(users)

In [14]:
counts = pd.DataFrame(counts)

In [15]:
users.rename(columns = {'rating': 'user_review_count'}, inplace = True)

In [31]:
counts.rename(columns = {'rating': 'game_review_count'}, inplace = True)

In [19]:
df_counts = df.join(users, on = 'user')

In [34]:
df_counts = df_counts.join(counts, on = 'name')

In [35]:
df_counts.head()

,user,rating,name,user_review_count,game_review_count
0,Torsten,10.0,Pandemic,1358.0,96234
1,mitnachtKAUBO-I,10.0,Pandemic,395.0,96234
2,avlawn,10.0,Pandemic,1036.0,96234
3,Mike Mayer,10.0,Pandemic,716.0,96234
4,Mease19,10.0,Pandemic,379.0,96234


In [23]:
df_counts.shape

(15823269, 4)

In [57]:
reduced_df = df_counts[(df_counts['game_review_count'] > 2000) & (df_counts['user_review_count'] >= 10)]

In [58]:
reduced_df.shape

(10954903, 5)

In [59]:
reduced_df['name'].nunique()

1424

In [60]:
reduced_df['user'].nunique()

191706

In [61]:
piv_df = pd.pivot_table(reduced_df, index = 'name', columns='user', values='rating')

In [62]:
piv_df.head()

user,mycroft,-=Yod@=-,-Johnny-,-Loren-,-LucaS-,-mIDE-,-mik-,-pj-,-snarf-,-toni-,...,zzz2525,zzzabiss,zzzeagle,zzzk1,zzzkardel,zzzuzu,zzzvone,zzzxxxyyy,zzzzzane,Æleksandr Þræð
name,,,,,,,,,,,,,,,,,,,,,
13 Days: The Cuban Missile Crisis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1775: Rebellion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1812: The Invasion of Canada,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1830: Railways & Robber Barons,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1960: The Making of the President,NaN,NaN,6.0,NaN,NaN,8.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
piv_df.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
Index: 1424 entries, 13 Days: The Cuban Missile Crisis to ZÈRTZ
Columns: 191706 entries,  mycroft to Æleksandr Þræð
dtypes: float64(191706)
memory usage: 2.0+ GB


In [66]:
sparse_df = sparse.csr_matrix(piv_df.fillna(0))

In [67]:
recommender = pairwise_distances(sparse_df, metric = 'cosine')

In [68]:
rec_df = pd.DataFrame(recommender, columns = piv_df.index, index = piv_df.index)

In [69]:
rec_df.head()

name,13 Days: The Cuban Missile Crisis,1775: Rebellion,1812: The Invasion of Canada,1830: Railways & Robber Barons,1960: The Making of the President,221B Baker Street: The Master Detective Game,5-Minute Dungeon,51st State,51st State: Master Set,6 nimmt!,...,Zombicide Season 2: Prison Outbreak,Zombicide Season 3: Rue Morgue,Zombicide: Black Plague,Zombicide: Green Horde,Zombie Dice,Zombie Fluxx,Zombie in my Pocket,Zombies!!!,Zooloretto,ZÈRTZ
name,,,,,,,,,,,,,,,,,,,,,
13 Days: The Cuban Missile Crisis,0.000000,0.811559,0.868826,0.901087,0.811312,0.960538,0.944378,0.924358,0.889856,0.861122,...,0.956232,0.967066,0.925853,0.966584,0.929983,0.970142,0.955841,0.943767,0.899327,0.924024
1775: Rebellion,0.811559,0.000000,0.671585,0.899418,0.816623,0.962115,0.960150,0.920266,0.910276,0.889128,...,0.949444,0.967822,0.930989,0.972180,0.920120,0.962221,0.948973,0.936028,0.881492,0.925614
1812: The Invasion of Canada,0.868826,0.671585,0.000000,0.895324,0.827162,0.964383,0.969862,0.907558,0.947905,0.896569,...,0.955566,0.970769,0.948403,0.979949,0.926574,0.963438,0.939938,0.927367,0.874613,0.911418
1830: Railways & Robber Barons,0.901087,0.899418,0.895324,0.000000,0.818172,0.946240,0.973216,0.901621,0.947453,0.850480,...,0.967795,0.978584,0.959942,0.982744,0.949839,0.963554,0.958470,0.933746,0.861737,0.879035
1960: The Making of the President,0.811312,0.816623,0.827162,0.818172,0.000000,0.932633,0.966515,0.857211,0.921407,0.822813,...,0.957425,0.969494,0.946355,0.980895,0.914043,0.937982,0.921286,0.888168,0.768802,0.845614


In [71]:
1 - rec_df['Pandemic'].sort_values()[:11]

name
Pandemic          1.000000
Carcassonne       0.583832
7 Wonders         0.581104
Dominion          0.571116
Catan             0.568363
Ticket to Ride    0.554892
Codenames         0.535606
Small World       0.531974
King of Tokyo     0.522275
Love Letter       0.508853
Splendor          0.505514
Name: Pandemic, dtype: float64

In [83]:
1 - rec_df['XCOM: The Board Game'].sort_values()[:11]

name
XCOM: The Board Game                                 1.000000
Dead of Winter: A Crossroads Game                    0.255387
Battlestar Galactica: The Board Game                 0.241412
Star Wars: Imperial Assault                          0.229693
Star Wars: Rebellion                                 0.224869
Eldritch Horror                                      0.222351
T.I.M.E Stories                                      0.220177
Robinson Crusoe: Adventures on the Cursed Island     0.218207
Legendary Encounters: An Alien Deck Building Game    0.216495
Mansions of Madness: Second Edition                  0.214675
Lords of Waterdeep                                   0.213760
Name: XCOM: The Board Game, dtype: float64

In [235]:
rec_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1424 entries, 13 Days: The Cuban Missile Crisis to ZÈRTZ
Columns: 1424 entries, 13 Days: The Cuban Missile Crisis to ZÈRTZ
dtypes: float64(1424)
memory usage: 15.5+ MB


In [236]:
rec_df.to_csv('./data/rec_df.csv')

In [238]:
info_df = pd.read_csv('./data/games_detailed_info.csv')

In [239]:
info_df.drop(columns=['Unnamed: 0', 'id', 'alternate', 'image', 'description', 'suggested_language_dependence', 'minplaytime', 'maxplaytime', 
                     'minage', 'boardgamefamily', 'boardgameexpansion', 'boardgameimplementation',
       'boardgamedesigner', 'boardgameartist', 'boardgamepublisher', 'usersrated', 'bayesaverage', 'Board Game Rank',
       'Strategy Game Rank', 'Family Game Rank', 'stddev', 'median', 'owned',
       'trading', 'wanting', 'wishing', 'numcomments', 'numweights',
       'averageweight', 'boardgameintegration', 'boardgamecompilation',
       'Party Game Rank', 'Abstract Game Rank', 'Thematic Rank',
       'War Game Rank', 'Customizable Rank', "Children's Game Rank",
       'RPG Item Rank', 'Accessory Rank', 'Video Game Rank', 'Amiga Rank',
       'Commodore 64 Rank', 'Arcade Rank', 'Atari ST Rank', 'suggested_num_players', 'suggested_playerage', 'type'], inplace=True)

In [240]:
info_df.rename(columns={'primary': 'name', 'thumbnail': 'Thumbnail', 'yearpublished': 'Year Published', 'minplayers': 'Min Players', 'maxplayers': 'Max Players',
                       'playingtime': 'Play Time (Minutes)', 'boardgamecategory': 'Category', 'boardgamemechanic': 'Mechanics', 'average': 'Average Rating'}, inplace = True)

In [241]:
info_df.set_index('name', inplace=True)

In [251]:
info_df['Average Rating'] = info_df['Average Rating'] / 10

In [252]:
info_df.head()

,Thumbnail,Year Published,Min Players,Max Players,Play Time (Minutes),Category,Mechanics,Average Rating
name,,,,,,,,
Pandemic,https://cf.geekdo-images.com/thumb/img/HEKrtpTC1y1amXh5cKnVvowyE5Y=/fit-in/200x150/pic1534148.jpg,2008,2,4,45,['Medical'],"['Action Points', 'Cooperative Game', 'Hand Management', 'Point to Point Movement', 'Set Collection', 'Trading', 'Variable Player Powers']",0.761567
Carcassonne,https://cf.geekdo-images.com/thumb/img/kqE4YJSm5PVkocYGl-V_IJcZQw4=/fit-in/200x150/pic2337577.jpg,2000,2,5,45,"['City Building', 'Medieval', 'Territory Building']","['Area Majority / Influence', 'Map Addition', 'Tile Placement']",0.741884
Catan,https://cf.geekdo-images.com/thumb/img/g8LvJsd2oLAub6AEKN_Xpdoi8Sw=/fit-in/200x150/pic2419375.jpg,1995,3,4,120,"['Economic', 'Negotiation']","['Dice Rolling', 'Hexagon Grid', 'Income', 'Modular Board', 'Network and Route Building', 'Race', 'Random Production', 'Trading', 'Variable Setup']",0.716265
7 Wonders,https://cf.geekdo-images.com/thumb/img/Grz-qM9xwxlvQGK7B-MiljtO9pQ=/fit-in/200x150/pic860217.jpg,2010,2,7,30,"['Ancient', 'Card Game', 'City Building', 'Civilization', 'Economic']","['Card Drafting', 'Drafting', 'Hand Management', 'Set Collection', 'Simultaneous Action Selection', 'Variable Player Powers']",0.776049
Dominion,https://cf.geekdo-images.com/thumb/img/iPlTR5cXFGrDuhupUgWGPjFjDPo=/fit-in/200x150/pic394356.jpg,2008,2,4,30,"['Card Game', 'Medieval']","['Deck, Bag, and Pool Building', 'Delayed Purchase', 'Hand Management', 'Take That', 'Variable Setup']",0.762671


In [253]:
info_df.to_csv('./data/info_df.csv')

In [254]:
temp_df = 1 - rec_df['Spartacus: A Game of Blood and Treachery'].sort_values()[1:11]

In [255]:
temp_df = pd.DataFrame(temp_df)

In [256]:
temp_df.rename(columns={'Spartacus: A Game of Blood and Treachery': 'Score'}, inplace = True)

In [257]:
temp_df.columns

Index(['Score'], dtype='object')

In [258]:
temp_df.head()

,Score
name,
Battlestar Galactica: The Board Game,0.238424
A Game of Thrones: The Board Game (Second Edition),0.236198
Cyclades,0.233905
Chaos in the Old World,0.231561
Blood Bowl: Team Manager – The Card Game,0.217015


In [259]:
temp_df.join(info_df)

,Score,Thumbnail,Year Published,Min Players,Max Players,Play Time (Minutes),Category,Mechanics,Average Rating
name,,,,,,,,,
A Game of Thrones: The Board Game (Second Edition),0.236198,https://cf.geekdo-images.com/thumb/img/jKVCyVKboWKMpLlI9pB9ojeqM_w=/fit-in/200x150/pic1077906.jpg,2011,3,6,240,"['Bluffing', 'Fantasy', 'Negotiation', 'Novel-based', 'Political', 'Wargame']","['Area Majority / Influence', 'Area Movement', 'Auction/Bidding', 'Auction: Sealed Bid', 'Card Play Conflict Resolution', 'Hand Management', 'Order Counters', 'Player Elimination', ""Prisoner's Dilemma"", 'Simultaneous Action Selection', 'Variable Player Powers']",0.761817
Battlestar Galactica: The Board Game,0.238424,https://cf.geekdo-images.com/thumb/img/NpZjJd2NgxSJV2WrlB_U1e89txY=/fit-in/200x150/pic354500.jpg,2008,3,6,180,"['Bluffing', 'Deduction', 'Movies / TV / Radio theme', 'Political', 'Science Fiction', 'Space Exploration', 'Spies/Secret Agents']","['Area Movement', 'Dice Rolling', 'Hand Management', 'Hidden Roles', 'Once-Per-Game Abilities', 'Role Playing', 'Team-Based Game', 'Traitor Game', 'Variable Player Powers', 'Variable Setup', 'Voting']",0.774223
Blood Bowl: Team Manager – The Card Game,0.217015,https://cf.geekdo-images.com/thumb/img/t1fhkbVdnXttA2WZJl9OO8RtbHk=/fit-in/200x150/pic1222746.jpg,2011,2,4,120,"['Card Game', 'Fantasy', 'Fighting', 'Sports']","['Area Majority / Influence', 'Card Drafting', 'Deck, Bag, and Pool Building', 'Dice Rolling', 'Hand Management']",0.738367
Blood Rage,0.209887,https://cf.geekdo-images.com/thumb/img/JPo2K0kSzYwCSqgEFdJbCzXFaTA=/fit-in/200x150/pic2439223.jpg,2015,2,4,90,"['Fantasy', 'Fighting', 'Mythology']","['Action Points', 'Area Majority / Influence', 'Area Movement', 'Card Drafting', 'Hand Management', 'Memory', 'Simultaneous Action Selection']",0.800234
Chaos in the Old World,0.231561,https://cf.geekdo-images.com/thumb/img/DB1gPPqF33aG34UT-3qCRt7A6IA=/fit-in/200x150/pic1318481.jpg,2009,3,4,120,"['Fantasy', 'Fighting', 'Horror', 'Mythology', 'Wargame']","['Action Points', 'Area Majority / Influence', 'Area Movement', 'Dice Rolling', 'Hand Management', 'Role Playing', 'Take That', 'Variable Player Powers']",0.767602
Cyclades,0.233905,https://cf.geekdo-images.com/thumb/img/LzbqfgQxxGqe4OlNM8sxBR8rYd8=/fit-in/200x150/pic584779.jpg,2009,2,5,90,"['Ancient', 'City Building', 'Civilization', 'Fighting', 'Miniatures', 'Mythology', 'Nautical']","['Area Majority / Influence', 'Area Movement', 'Auction/Bidding', 'Auction: Fixed Placement', 'Card Drafting', 'Dice Rolling', 'Set Collection', 'Variable Phase Order']",0.752205
Dead of Winter: A Crossroads Game,0.206986,https://cf.geekdo-images.com/thumb/img/da35zyUhqdAVUoYNFvTdvp_MsnY=/fit-in/200x150/pic3016500.jpg,2014,2,5,120,"['Bluffing', 'Deduction', 'Horror', 'Zombies']","['Action Points', 'Area Movement', 'Cooperative Game', 'Dice Rolling', 'Hand Management', 'Narrative Choice / Paragraph', 'Push Your Luck', 'Semi-Cooperative Game', 'Storytelling', 'Trading', 'Traitor Game', 'Variable Player Powers', 'Voting']",0.759954
Kemet,0.209070,https://cf.geekdo-images.com/thumb/img/_7ADXwUAZ59PuJbKoXOD5a146vE=/fit-in/200x150/pic3979527.jpg,2012,2,5,120,"['Ancient', 'Fighting', 'Miniatures', 'Mythology', 'Wargame']","['Action Points', 'Area Majority / Influence', 'Area Movement', 'Card Drafting', 'Card Play Conflict Resolution', 'Hand Management']",0.772289
Merchants & Marauders,0.209446,https://cf.geekdo-images.com/thumb/img/frmqzXc01uvqO6usycue7Tnz8xI=/fit-in/200x150/pic738119.jpg,2010,2,4,180,"['Adventure', 'Fighting', 'Nautical', 'Pirates', 'Transportation']","['Action Points', 'Area Movement', 'Dice Rolling', 'Pick-up and Deliver', 'Race', 'Variable Player Powers']",0.742219
